# Вебинар 5. Ранжирование и Гибридные рекоммендательные системы

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [8]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [10]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [11]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [12]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [13]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [14]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [15]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [16]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [17]:

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [18]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [19]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [20]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
user_feat.shape

(2497, 7)

In [22]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [23]:
item_feat.shape

(5001, 6)

## Encoding features

In [24]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [25]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

## Model functions

In [28]:
def init_model(learning_rate=0.01, item_alpha=0.4, user_alpha=0.1):
    model = LightFM(no_components=60,
                    loss='bpr', # "logistic","bpr"
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    user_alpha=user_alpha,
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
    return model

def fit_model(model):
    return model.fit((sparse_user_item > 0) * 1,
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20,
              num_threads=20,
              verbose=True)

def predict_model(model):
    predictions = model.predict(user_ids=users_ids_row,
                                item_ids=items_ids_row,
                                user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                num_threads=10)
    return predictions

def update_predict_results(predictions, df_result_for_metrics):
    
    data_train_filtered['score'] = predictions

    
    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id'].unique().reset_index()

    
    df_result_for_metrics = df_result_for_metrics.merge(predict_result, on='user_id', how='inner')

    return df_result_for_metrics

## Grid Search

In [29]:
def grid_search(learning_rate=None, item_alpha=None, user_alpha=None):
    if learning_rate is None:
        learning_rate = [0.01]
    if item_alpha is None:
        item_alpha = [0.4]
    if user_alpha is None:
        user_alpha = [0.1]
    df_result_for_metrics = result
    predict_results = pd.DataFrame()
    for learning_rate_ in learning_rate:
        for item_alpha_ in item_alpha:
            for user_alpha_ in user_alpha:

                model = init_model(learning_rate=learning_rate_, item_alpha=item_alpha_, user_alpha=user_alpha_)
                fit_model(model)
                predictions = predict_model(model)

                df_result_for_metrics = update_predict_results(predictions, df_result_for_metrics)

                precision = round(df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean(), 4)

                kol_name = f'learning_rate: {learning_rate_} item_alpha: {item_alpha_} user_alpha: {user_alpha_}'
                df_result_for_metrics = df_result_for_metrics.rename(columns={'item_id': kol_name})

                predict_results = predict_results.append(pd.DataFrame([precision], columns=['precision'], index=[kol_name]))

                print(f"Precision: {precision} | {kol_name}")

    predict_results = predict_results.sort_values(by=['precision'], ascending=False)
    best = f'Best parameters: {predict_results.head(1).precision.tolist()[0]}, {predict_results.head(1).index.tolist()[0]}'
    print(f'\n{best}')

    return df_result_for_metrics, predict_results, best

In [30]:
all_results, predict_results, best = grid_search(learning_rate = [0.01, 0.05],
                          item_alpha = [0.4, 0.3],
                          user_alpha = [0.1, 0.2])

Epoch: 100%|████████████████████████████████████| 20/20 [01:00<00:00,  3.01s/it]


Precision: 0.0621 | learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.1


Epoch: 100%|████████████████████████████████████| 20/20 [01:01<00:00,  3.07s/it]


Precision: 0.1387 | learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.2


Epoch: 100%|████████████████████████████████████| 20/20 [01:01<00:00,  3.07s/it]


...

Precision: 0.0558 | learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.2


Epoch: 100%|████████████████████████████████████| 20/20 [01:00<00:00,  3.01s/it]


Precision: 0.1379 | learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.1


Epoch: 100%|████████████████████████████████████| 20/20 [00:59<00:00,  2.97s/it]


Precision: 0.0549 | learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.2

Best parameters: 0.1408, learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.1


In [31]:
predict_results

,precision
learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.1,0.1408
learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.2,0.1387
learning_rate: 0.01 item_alpha: 0.3 user_alpha: 0.2,0.1387
learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.1,0.1379
learning_rate: 0.01 item_alpha: 0.4 user_alpha: 0.1,0.0621
learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.1,0.0564
learning_rate: 0.05 item_alpha: 0.4 user_alpha: 0.2,0.0558
learning_rate: 0.05 item_alpha: 0.3 user_alpha: 0.2,0.0549
